In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)
import glob
import lightgbm as lgbm
from sklearn.model_selection import KFold
import pickle

import os
import sys
sys.path.append('..')
data_dir = '../input/optiver-realized-volatility-prediction/'
from preprocess import preprocessor_book

In [31]:
book_df = pd.read_parquet(data_dir+"book_train.parquet")
trade_df = pd.read_parquet(data_dir+"trade_train.parquet")

欠損値確認

In [32]:
book_df.isnull().sum()

time_id              0
seconds_in_bucket    0
bid_price1           0
ask_price1           0
bid_price2           0
ask_price2           0
bid_size1            0
ask_size1            0
bid_size2            0
ask_size2            0
stock_id             0
dtype: int64

In [33]:
trade_df.isnull().sum()

time_id              0
seconds_in_bucket    0
price                0
size                 0
order_count          0
stock_id             0
dtype: int64

欠損値があるわけではなく、bookデータで抜けてるseconds_in_bucketがあるということ

In [34]:
book_df

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100,0
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0
...,...,...,...,...,...,...,...,...,...,...,...
167253284,32767,589,0.999662,0.999797,0.999527,0.999932,400,600,400,528,99
167253285,32767,590,0.999662,0.999797,0.999527,0.999932,400,500,400,528,99
167253286,32767,592,0.999527,0.999797,0.999391,0.999932,900,600,600,528,99
167253287,32767,596,0.999527,0.999797,0.999391,0.999932,900,700,600,628,99


In [35]:
def ffill(data_df):
    data_df=data_df.set_index(['time_id', 'seconds_in_bucket'])
    data_df = data_df.reindex(pd.MultiIndex.from_product([data_df.index.levels[0], np.arange(0,600)], names = ['time_id', 'seconds_in_bucket']), method='ffill')
    return data_df.reset_index()

In [36]:
book_df[(book_df['stock_id'] == 0) & (book_df['time_id'] == 5)]

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100,0
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100,0
...,...,...,...,...,...,...,...,...,...,...,...
297,5,585,1.003129,1.003749,1.003025,1.003801,100,3,26,3,0
298,5,586,1.003129,1.003749,1.002612,1.003801,100,3,2,3,0
299,5,587,1.003129,1.003749,1.003025,1.003801,100,3,26,3,0
300,5,588,1.003129,1.003749,1.002612,1.003801,100,3,2,3,0


In [37]:
book_df = ffill(book_df[book_df['stock_id'] == 0])

In [38]:
book_df[(book_df['stock_id'] == 0) & (book_df['time_id'] == 5)]

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0
2,5,2,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0
3,5,3,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0
4,5,4,1.001422,1.002301,1.001370,1.002353,3,100,2,100,0
...,...,...,...,...,...,...,...,...,...,...,...
595,5,595,1.003129,1.003749,1.003025,1.003801,100,3,26,3,0
596,5,596,1.003129,1.003749,1.003025,1.003801,100,3,26,3,0
597,5,597,1.003129,1.003749,1.003025,1.003801,100,3,26,3,0
598,5,598,1.003129,1.003749,1.003025,1.003801,100,3,26,3,0


600秒分埋めてくれる！！

In [39]:
preprocessor_book(data_dir+"book_train.parquet/stock_id=0")

,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_150,log_return_mean_150,log_return_std_150,log_return_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return2_sum_150,wap_balance_mean_150,wap_balance_std_150,wap_balance_sum_150,price_spread_mean_150,price_spread_std_150,price_spread_sum_150,bid_spread_mean_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,ask_spread_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,volume_imbalance_sum_150,total_volume_mean_150,total_volume_std_150,total_volume_sum_150,wap_mean_150,wap_std_150,wap_sum_150,row_id
0,0.004499,3.825865e-06,0.000184,0.002292,0.006999,3.881845e-06,0.000286,0.002325,0.000394,0.000303,0.236108,0.000865,0.000212,0.518973,0.000183,0.000166,0.109890,-0.000132,0.000111,-0.078963,140.206667,110.228465,84124,312.733333,137.593491,187640,1.003690,0.000702,602.214218,0.001721,-2.409433e-06,0.000141,-0.000361,0.004114,4.537230e-07,0.000337,0.000068,0.000322,0.000272,0.048311,0.000771,0.000177,0.115701,0.000232,0.000177,0.034752,-0.000128,0.000112,-0.019135,163.020000,80.862587,24453,252.193333,106.508890,37829,1.003555,0.000558,150.533267,0.002953,5.242750e-07,0.000171,0.000157,0.004863,9.121772e-07,0.000281,0.000274,0.000356,0.000269,0.106717,0.000832,0.000237,0.249618,0.000217,0.000169,0.064952,-0.000129,0.000113,-0.038580,151.156667,95.469683,45347,282.430000,133.923324,84729,1.003784,0.000496,301.135251,0.003796,6.133827e-07,0.000179,0.000276,0.006087,6.678874e-09,0.000287,0.000003,0.000374,0.000270,0.168150,0.000863,0.000223,0.388178,0.000196,0.000171,0.088377,-0.000129,0.000109,-0.058172,135.495556,106.904871,60973,316.917778,141.381412,142613,1.003833,0.000457,451.724786,0-5
1,0.001204,6.013981e-07,0.000049,0.000360,0.002476,1.336806e-06,0.000101,0.000801,0.000206,0.000143,0.123368,0.000432,0.000185,0.259401,0.000131,0.000131,0.078852,-0.000138,0.000071,-0.082809,129.110000,94.455038,77466,380.630000,172.053743,228378,1.000184,0.000263,600.110544,0.000918,-3.936326e-07,0.000075,-0.000059,0.001883,3.252438e-06,0.000154,0.000488,0.000259,0.000140,0.038898,0.000386,0.000164,0.057944,0.000179,0.000210,0.026803,-0.000138,0.000066,-0.020727,98.300000,96.763099,14745,415.446667,189.278054,62317,1.000471,0.000218,150.070666,0.000981,3.21377